Version 1 : 3 methods (RAG + DP + Q_learning)

In [ ]:
from typing import List
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

items = [
    "North Indian Thali", "Phulkha (3 pcs)", "Chapathi (2 pcs)", "Roti", "Naan", 
    "Butter Roti", "Butter Naan", "Uthapam", "Plain Dosa", "Onion Dosa", "Onion Uthapam", 
    "Masala Dosa", "Plain Rice", "Jeera Rice", "Veg Noodles", "Egg Noodles", 
    "Chicken Noodles", "Moong Dhal", "Aloo Pakke Subji", "Black Channa Masala", 
    "Chilly Baby Corn", "Papedi Chat", "Pani Poori", "Dahi Poori", "Veg Rice", 
    "Egg Rice", "Chicken Rice", "Mushroom Rice", "Egg Biryani", "Chicken Biryani", 
    "Chicken 65", "Egg Curry", "Single Egg Curry", "Boiled Eggs", "Scrambled Eggs", 
    "Pepper Chicken", "Chicken Masala", "Chicken Manchurian", "Lemon Juice", "Watermelon Juice"
]

costs = [
    68, 16, 14, 9, 9, 14, 14, 24, 21, 27, 27, 32, 21, 34, 45, 63, 66, 27, 32, 39, 
    43, 21, 17, 21, 47, 61, 76, 56, 63, 83, 78, 23, 13, 9, 13, 70, 68, 72, 13, 19
]

Meals = {}
for item, cost in zip(items, costs):
    Meals[item] = {"cost": cost, "rating": 5}

# Learning rate
alpha = 0.1
#Exploration-Exploitation tradeoff
epsilon = 0.1

Q_table = {meal: 5 for meal in Meals}

meal_texts = [" ".join(meal.split()) for meal in Meals]
vectorizer = TfidfVectorizer()
meal_vectors = vectorizer.fit_transform(meal_texts)


def RAG_similar_meals(preferred_meal: str, top_n: int = 3) -> List[str]:
    "Get similar meals using TFID and cosine similarity"
    # process preferred meal
    text = " ".join(preferred_meal.split())
    
    #transform into vector
    vector = vectorizer.transform([text])
    
    # calculate similarity
    similarities = cosine_similarity(vector, meal_vectors).flatten()
    
    # sort based on similarity score
    sorted_indices = np.argsort(similarities)[::-1]
    
    # Get the top_n meals
    similar_indices = sorted_indices[-(top_n+1):-1]
    similar_indices = similar_indices[::-1] # reverse
    
    # convert dict to list for index access
    similar_meals = []
    meal_keys = list(Meals.keys())
    
    for index in similar_indices:
        similar_meals.append(meal_keys[index])
        
    return similar_meals

def knapsack_dp(meals_list : List[str], budget : int) -> List[str]:
    "Select the best combination of meals using Knapsack DP"
    
    n = len(meals_list)
    # Initialize DP table with 0s in a matrix of size (n+1)*(budget+1)
    dp = [[0] * (budget + 1) for _ in range(n + 1)]  

    # Select meals
    for i in range(1,n+1):
        current_meal = meals_list[i-1]
        cost = Meals[current_meal]["cost"]
        rating = Q_table[current_meal]
        
        for j in range(budget+1):
            if cost <= j:
                dp[i][j] = max(dp[i-1][j], dp[i-1][j-cost] + rating) 
            else:
                dp[i][j] = dp[i-1][j]
                
    # Backtrack to find the selected meals
    selected_meals = []
    j = budget
    for i in range(n, 0, -1):   
        if dp[i][j] != dp[i-1][j]:
            selected_meals.append(meals_list[i-1])
            j -= Meals[meals_list[i-1]]["cost"]
            
    return selected_meals    
    
def Q_learning_update(meal: str, rating: int):
    "Update the Q-table based on user feedback"
    Q_table[meal] = Q_table[meal] + alpha * (rating - Q_table[meal]) # rating = reward

def hybrid_recommend(preferences: List[str], dislikes: List[str], budget: int) -> List[str]:
    "Combination of the 3 methods"
    # Step 1 - Exploration vs Exploitation
    if np.random.rand() < epsilon:
        # Explore: Randomly select meals not in disliked list
        candidate_meals = [meal for meal in Meals if meal not in dislikes]
    else:
        # Exploit: Select meals with highest Q_values
        candidate_meals = sorted(Q_table, key=Q_table.get, reverse=True)
        candidate_meals = [meal for meal in candidate_meals if meal not in dislikes]
    
    # Step 2 - Knapsack budget constraint
    selected_meals = knapsack_dp(candidate_meals, budget)
    
    # Step 3 - RAG enhanced
    for meal in selected_meals:
        similar_meals = RAG_similar_meals(meal)
        print(f"Since you liked {meal}, you might also like: {', '.join(similar_meals)}")
    
    return selected_meals

Version 2: RAG removed cause it s similarity is semantic while we require custom values 

In [ ]:
from typing import List
import numpy as np

items = [
    "North Indian Thali", "Phulkha (3 pcs)", "Chapathi (2 pcs)", "Roti", "Naan", 
    "Butter Roti", "Butter Naan", "Uthapam", "Plain Dosa", "Onion Dosa", "Onion Uthapam", 
    "Masala Dosa", "Plain Rice", "Jeera Rice", "Veg Noodles", "Egg Noodles", 
    "Chicken Noodles", "Moong Dhal", "Aloo Pakke Subji", "Black Channa Masala", 
    "Chilly Baby Corn", "Papedi Chat", "Pani Poori", "Dahi Poori", "Veg Rice", 
    "Egg Rice", "Chicken Rice", "Mushroom Rice", "Egg Biryani", "Chicken Biryani", 
    "Chicken 65", "Egg Curry", "Single Egg Curry", "Boiled Eggs", "Scrambled Eggs", 
    "Pepper Chicken", "Chicken Masala", "Chicken Manchurian", "Lemon Juice", "Watermelon Juice"
]

costs = [
    68, 16, 14, 9, 9, 14, 14, 24, 21, 27, 27, 32, 21, 34, 45, 63, 66, 27, 32, 39, 
    43, 21, 17, 21, 47, 61, 76, 56, 63, 83, 78, 23, 13, 9, 13, 70, 68, 72, 13, 19
]

Meals = {}
for item, cost in zip(items, costs):
    Meals[item] = {"cost": cost, "rating": 5}

# Learning rate
alpha = 0.1
# Exploration-Exploitation tradeoff
epsilon = 0.1

Q_table = {meal: 5 for meal in Meals}

def knapsack_dp(meals_list: List[str], budget: int) -> List[str]:
    """Select the best combination of meals using Knapsack DP"""
    n = len(meals_list)
    dp = [[0] * (budget + 1) for _ in range(n + 1)]

    for i in range(1, n + 1):
        current_meal = meals_list[i-1]
        cost = Meals[current_meal]["cost"]
        rating = Q_table[current_meal]
        
        for j in range(budget + 1):
            if cost <= j:
                dp[i][j] = max(dp[i-1][j], dp[i-1][j - cost] + rating)
            else:
                dp[i][j] = dp[i-1][j]

    selected_meals = []
    j = budget
    for i in range(n, 0, -1):
        if dp[i][j] != dp[i-1][j]:
            selected_meals.append(meals_list[i-1])
            j -= Meals[meals_list[i-1]]["cost"]
            
    return selected_meals

def Q_learning_update(meal: str, rating: int):
    """Update the Q-table based on user feedback"""
    Q_table[meal] = Q_table[meal] + alpha * (rating - Q_table[meal])

def hybrid_recommend(preferences: List[str], dislikes: List[str], budget: int) -> List[str]:
    """Combination of Q-learning and Knapsack DP"""
    if np.random.rand() < epsilon:
        candidate_meals = [meal for meal in Meals if meal not in dislikes]
    else:
        candidate_meals = sorted(Q_table, key=Q_table.get, reverse=True)
        candidate_meals = [meal for meal in candidate_meals if meal not in dislikes]
    
    selected_meals = knapsack_dp(candidate_meals, budget)
    return selected_meals

def test_harness():
    """Test the recommendation system with simulated user interactions"""
    test_cases = [
        {"preferences": ["Chicken Biryani"], "dislikes": ["Boiled Eggs"], "budget": 200},
        {"preferences": ["Veg Noodles"], "dislikes": ["Egg Curry"], "budget": 150},
        {"preferences": ["Masala Dosa"], "dislikes": ["Chicken 65"], "budget": 100}
    ]

    for i, test in enumerate(test_cases, 1):
        print(f"\n{'='*40}\n Test Case {i}: Budget ₹{test['budget']}, Likes: {test['preferences']}, Dislikes: {test['dislikes']}\n{'='*40}")

        recommendations = hybrid_recommend(
            preferences=test['preferences'],
            dislikes=test['dislikes'],
            budget=test['budget']
        )

        total_cost = sum(Meals[meal]['cost'] for meal in recommendations)
        print(f"\nRecommended meals: {recommendations}")
        print(f"Total cost: ₹{total_cost}/₹{test['budget']}")

        if recommendations:
            liked_meal = np.random.choice(recommendations)
            Q_learning_update(liked_meal, rating=9)
            print(f"\nUser liked '{liked_meal}' → Updated Q-values:")
            print({k: round(v, 2) for k, v in Q_table.items() if k in recommendations})

def main():
    """Main execution block"""
    np.random.seed(42)
    print("🍽️ Meal Recommendation System Test Harness 🍴\n")
    test_harness()
    print("\n✅ Testing completed!")

if __name__ == "__main__":
    main()


Version 1.3 :  Similarity Matrix introduced

In [ ]:
from typing import List
import numpy as np

items = [
    "North Indian Thali", "Phulkha (3 pcs)", "Chapathi (2 pcs)", "Roti", "Naan", 
    "Butter Roti", "Butter Naan", "Uthapam", "Plain Dosa", "Onion Dosa", "Onion Uthapam", 
    "Masala Dosa", "Plain Rice", "Jeera Rice", "Veg Noodles", "Egg Noodles", 
    "Chicken Noodles", "Moong Dhal", "Aloo Pakke Subji", "Black Channa Masala", 
    "Chilly Baby Corn", "Papedi Chat", "Pani Poori", "Dahi Poori", "Veg Rice", 
    "Egg Rice", "Chicken Rice", "Mushroom Rice", "Egg Biryani", "Chicken Biryani", 
    "Chicken 65", "Egg Curry", "Single Egg Curry", "Boiled Eggs", "Scrambled Eggs", 
    "Pepper Chicken", "Chicken Masala", "Chicken Manchurian", "Lemon Juice", "Watermelon Juice"
]

costs = [
    68, 16, 14, 9, 9, 14, 14, 24, 21, 27, 27, 32, 21, 34, 45, 63, 66, 27, 32, 39, 
    43, 21, 17, 21, 47, 61, 76, 56, 63, 83, 78, 23, 13, 9, 13, 70, 68, 72, 13, 19
]

# --- MEAL DATA STRUCTURE ---
Meals = {item: {"cost": cost, "rating": 5} for item, cost in zip(items, costs)}

# --- DIETARY CATEGORIES ---
veg_meals = [
    "North Indian Thali", "Phulkha (3 pcs)", "Chapathi (2 pcs)", "Roti", "Naan",
    "Butter Roti", "Butter Naan", "Uthapam", "Plain Dosa", "Onion Dosa",
    "Onion Uthapam", "Masala Dosa", "Moong Dhal", "Aloo Pakke Subji",
    "Black Channa Masala", "Chilly Baby Corn", "Papedi Chat", "Pani Poori",
    "Dahi Poori", "Lemon Juice", "Watermelon Juice", "Plain Rice", "Jeera Rice",
    "Veg Rice", "Mushroom Rice", "Veg Noodles"
]

nonveg_meals = [
    "Chicken Biryani", "Egg Biryani", "Chicken 65", "Egg Curry",
    "Single Egg Curry", "Pepper Chicken", "Chicken Masala", "Chicken Manchurian",
    "Egg Rice", "Chicken Rice", "Egg Noodles", "Chicken Noodles",
    "Boiled Eggs", "Scrambled Eggs"
]

# --- SIMILARITY MATRIX INITIALIZATION ---
n_meals = len(items)
meal_to_idx = {meal: i for i, meal in enumerate(items)}
similarity_matrix = np.zeros((n_meals, n_meals))

def set_relationship(meal_a: str, meal_b: str, value: float):
    """Set relationship value between two meals"""
    i, j = meal_to_idx[meal_a], meal_to_idx[meal_b]
    value = max(min(value, 1.0), -1.0)  # Clamp between -1 and 1
    similarity_matrix[i][j] = value
    similarity_matrix[j][i] = value  # Ensure symmetry

def initialize_relationships():
    """Automatically set initial relationships based on meal groups"""
    # Define meal groups
    groups = {
        "standalone": [
            "North Indian Thali", "Chicken Biryani", "Egg Biryani"
        ],
        "snacks_juices": [
            "Papedi Chat", "Pani Poori", "Dahi Poori", 
            "Lemon Juice", "Watermelon Juice"
        ],
        "breads": [
            "Phulkha (3 pcs)", "Chapathi (2 pcs)", "Roti", "Naan",
            "Butter Roti", "Butter Naan", "Uthapam", "Plain Dosa",
            "Onion Dosa", "Onion Uthapam", "Masala Dosa"
        ],
        "side_dishes": [
            "Moong Dhal", "Aloo Pakke Subji", "Black Channa Masala",
            "Chilly Baby Corn", "Chicken 65", "Egg Curry",
            "Single Egg Curry", "Pepper Chicken", "Chicken Masala",
            "Chicken Manchurian"
        ]
    }

    # Set alternatives within same group
    for group_meals in groups.values():
        for i in range(len(group_meals)):
            for j in range(i+1, len(group_meals)):
                set_relationship(group_meals[i], group_meals[j], -0.7)

    # Set complements between groups
    complementary_groups = [
        ("standalone", "snacks_juices"),
        ("breads", "side_dishes")
    ]
    
    for group1, group2 in complementary_groups:
        for meal1 in groups[group1]:
            for meal2 in groups[group2]:
                set_relationship(meal1, meal2, 0.6)

    # Set standalone incompatibility with other main components
    for standalone in groups["standalone"]:
        for meal in groups["breads"] + groups["side_dishes"]:
            set_relationship(standalone, meal, -0.8)

    # Set veg-nonveg incompatibility
    for veg in veg_meals:
        for nonveg in nonveg_meals:
            if veg != nonveg:
                set_relationship(veg, nonveg, -0.4)


initialize_relationships()

# --- LEARNING PARAMETERS ---
alpha = 0.1  # Learning rate
epsilon = 0.1  # Exploration rate
Q_table = {meal: 5 for meal in Meals}  # Initial Q-values
meal_history = []  # Track meal history for variety

# --- KNAPSACK DP IMPLEMENTATION ---
def knapsack_dp(meals_list: List[str], budget: int) -> List[str]:
    """Select optimal meals using similarity-aware Knapsack DP"""
    n = len(meals_list)
    dp = [[0] * (budget + 1) for _ in range(n + 1)]
    selected_in_step = [[] for _ in range(budget + 1)]

    for i in range(1, n + 1):
        current_meal = meals_list[i-1]
        cost = Meals[current_meal]["cost"]
        base_rating = Q_table[current_meal]
        
        for j in range(budget + 1):
            if cost <= j:
                # Calculate similarity adjustment
                similarity_boost = sum(
                    similarity_matrix[meal_to_idx[current_meal]][meal_to_idx[m]]
                    for m in selected_in_step[j - cost]
                )
                adjusted_rating = base_rating + similarity_boost
                
                if (dp[i-1][j - cost] + adjusted_rating) > dp[i-1][j]:
                    dp[i][j] = dp[i-1][j - cost] + adjusted_rating
                    selected_in_step[j] = selected_in_step[j - cost] + [current_meal]
                else:
                    dp[i][j] = dp[i-1][j]
            else:
                dp[i][j] = dp[i-1][j]

    return selected_in_step[budget]

# --- Q-LEARNING UPDATE ---
def Q_learning_update(meal: str, rating: int):
    """Update Q-values based on user feedback"""
    Q_table[meal] += alpha * (rating - Q_table[meal])

# --- HYBRID RECOMMENDATION SYSTEM ---
def hybrid_recommend(preferences: List[str], dislikes: List[str], 
                    budget: int, dietary: str) -> List[str]:
    """Combination of Q-learning and Knapsack DP with dietary filtering"""
    # Determine available meals
    available_meals = veg_meals if dietary == "veg" else nonveg_meals
    
    # Filter preferences and dislikes
    valid_prefs = [m for m in preferences if m in available_meals]
    valid_dislikes = [m for m in dislikes if m in available_meals]
    
    # Exploration vs Exploitation
    if np.random.rand() < epsilon:
        candidate_meals = [m for m in available_meals 
                          if m not in valid_dislikes and m not in meal_history]
    else:
        candidate_meals = sorted(
            [m for m in available_meals if m not in valid_dislikes],
            key=lambda x: Q_table[x], 
            reverse=True
        )[:len(available_meals)//2]  # Top 50% of available meals
    
    # Knapsack selection
    selected_meals = knapsack_dp(candidate_meals, budget)
    
    # Update meal history for variety
    meal_history.extend(selected_meals)
    if len(meal_history) > 15:  # Keep last 15 meals in history
        meal_history.pop(0)
    
    return selected_meals

# --- TEST HARNESS WITH DIETARY SELECTION ---
def test_harness():
    """Interactive test harness with dietary preference selection"""
    # Get dietary preference
    while True:
        dietary_choice = input("\nChoose dietary preference (veg/nonveg): ").lower()
        if dietary_choice in ["veg", "nonveg"]:
            break
        print("Invalid choice! Please enter 'veg' or 'nonveg'.")
    
    # Define test cases
    test_cases = [
        {"preferences": ["Chicken Biryani", "Egg Biryani"], 
         "dislikes": ["Boiled Eggs"], "budget": 200},
        {"preferences": ["Veg Noodles", "Jeera Rice"], 
         "dislikes": ["Egg Curry"], "budget": 150},
        {"preferences": ["Masala Dosa", "Uthapam"], 
         "dislikes": ["Chicken 65"], "budget": 100}
    ]
    
    # Filter test cases by dietary choice
    filtered_cases = []
    for case in test_cases:
        filtered_prefs = [m for m in case["preferences"] 
                         if m in (veg_meals if dietary_choice == "veg" else nonveg_meals)]
        filtered_dislikes = [m for m in case["dislikes"] 
                            if m in (veg_meals if dietary_choice == "veg" else nonveg_meals)]
        filtered_cases.append({
            "preferences": filtered_prefs,
            "dislikes": filtered_dislikes,
            "budget": case["budget"]
        })

    print(f"\n{'='*40}\n Testing {dietary_choice.capitalize()} Recommendations\n{'='*40}")
    
    for i, test in enumerate(filtered_cases, 1):
        print(f"\nTest Case {i}:")
        print(f"Budget: ₹{test['budget']}")
        print(f"Preferences: {test['preferences'] or 'None'}")
        print(f"Dislikes: {test['dislikes'] or 'None'}")
        
        recommendations = hybrid_recommend(
            preferences=test['preferences'],
            dislikes=test['dislikes'],
            budget=test['budget'],
            dietary=dietary_choice
        )
        
        total_cost = sum(Meals[meal]['cost'] for meal in recommendations)
        print(f"\nRecommended meals: {recommendations}")
        print(f"Total Cost: ₹{total_cost}/₹{test['budget']}")
        print(f"Budget Utilization: {(total_cost/test['budget'])*100:.1f}%")
        
        if recommendations:
            # Simulate user feedback
            liked_meal = np.random.choice(recommendations)
            Q_learning_update(liked_meal, 9)  # Assume user liked the meal
            print(f"\nUser liked '{liked_meal}' → Updated Q-value: {Q_table[liked_meal]:.2f}")

def main():
    np.random.seed(42)  # For reproducible results
    print("🍽️ Smart Meal Recommendation System 🧠")
    print("With Dietary Preference Filtering ⚡\n")
    test_harness()
    print("\n Test Completed!")

if __name__ == "__main__":
    main()
